In [52]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model

In [53]:
#Load the trained model, scaler and on hot encoder pickle file
model = load_model('model.h5')

#load encoder and scaler

with open('label_encoder.pkl','rb') as f:
    label_encoder_gender = pickle.load(f)

with open('one_hot_encoder.pkl','rb') as f:
    one_hot_geo = pickle.load(f)

with open('standard_scaler.pkl','rb') as f:
    scaler = pickle.load(f)



In [54]:
#Example prediction
# Create example input data
input_data = {
    'CreditScore': 650,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 35,
    'Tenure': 5,
    'Balance': 75000.0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 65000.0
}


In [55]:
#One Hot Encoding for Geography
geo_encoded = one_hot_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=['France', 'Germany', 'Spain'])
geo_encoded_df


c:\Users\mkuma\OneDrive\Desktop\ANN_FIRST_PROJECT\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,France,Germany,Spain
0,1.0,0.0,0.0


In [56]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,Female,35,5,75000.0,2,1,1,65000.0


In [57]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,0,35,5,75000.0,2,1,1,65000.0


In [58]:
#Concation of data
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [59]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,650,0,35,5,75000.0,2,1,1,65000.0,1.0,0.0,0.0


In [60]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.01709861, -1.09499335, -0.37056859, -0.00134472, -0.01764635,
         0.80843615,  0.64920267,  0.97481699, -0.61603215,  1.00150113,
        -0.57946723, -0.57638802]])

In [61]:
#Predicting output
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


array([[0.06694707]], dtype=float32)

In [63]:
prediction_proba = prediction[0][0]

In [64]:
prediction_proba

0.066947065

In [65]:
if prediction_proba > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
